In [ ]:
!pip install shapely geopandas

In [ ]:
import shapely
from shapely.geometry import Point
import geopandas as gpd
import pandas as pd
import numpy as np

In [ ]:
# Load country shapes
country_shapes = gpd.read_file("s3://w210-poverty-mapper/shape_files/raw_data/ipums_world_shapes/world_countries_2020.shp")

In [ ]:
country_codes = { "Bangladesh" : "BD",
                 "Nepal" : "NP",
                 "Philippines": "PH",
                 "Tajikistan" : "TJ",
                 "Pakistan" : "PK",
                 "Timor Leste": "TL"
                }

In [ ]:
dhs_data = pd.read_csv("s3://w210-poverty-mapper/dhs_data/transformed_data/dhs_wealth_index.csv")

In [ ]:
polygons = {}
for country in country_codes:
    country_code = country_codes[country]
    polygon = country_shapes.loc[country_shapes["CNTRY_NAME"] == country]["geometry"].iloc[0]    
    polygons[country_code] = polygon

In [ ]:
polygons

In [ ]:
coordinates = dhs_data.apply(lambda row: polygons[row["country"]].contains(Point(row["lon"], row["lat"])), axis=1)

In [ ]:
np.unique(coordinates, return_counts=True)

In [ ]:
dhs_data["inside_boundaries"] = coordinates

In [ ]:
dhs_data[dhs_data["inside_boundaries"] == False]

In [ ]:
dhs_data.to_csv("s3://w210-poverty-mapper/dhs_data/transformed_data/dhs_wealth_index_boundaries.csv", index=False)